In [1]:
#!pip install duckdb==1.2.2
#!pip install pandas

import pandas as pd
import duckdb

from google.colab import files

In [2]:
con = duckdb.connect()

duckdb.sql("CREATE TABLE patients AS SELECT * FROM read_csv_auto('PATIENTS.csv');")
duckdb.sql("CREATE TABLE admissions AS SELECT * FROM read_csv_auto('ADMISSIONS.csv');")
duckdb.sql("CREATE TABLE prescriptions AS SELECT * FROM read_csv_auto('PRESCRIPTIONS.csv');")
duckdb.sql("CREATE TABLE procedures_icd AS SELECT * FROM read_csv_auto('PROCEDURES_ICD.csv');")
duckdb.sql("CREATE TABLE d_icd_procedures AS SELECT * FROM read_csv_auto('D_ICD_PROCEDURES.csv');")
duckdb.sql("CREATE TABLE icustays AS SELECT * FROM read_csv_auto('ICUSTAYS.csv');")
duckdb.sql("CREATE TABLE drgcodes AS SELECT * FROM read_csv_auto('DRGCODES.csv');")

In [3]:
con.sql("SHOW TABLES;").df()
admissions = pd.read_csv("ADMISSIONS.csv")
prescriptions = pd.read_csv("PRESCRIPTIONS.csv")
con.register("admissions", admissions)
con.register("prescriptions", prescriptions)

In [4]:
ethnicity_drug_summary = con.execute("""
    SELECT
        a.ethnicity AS Ethnicity,
        p.drug AS Drug,
        SUM(CAST(p.dose_val_rx AS DOUBLE)) AS TotalDose
    FROM prescriptions p
    JOIN admissions a ON p.hadm_id = a.hadm_id
    WHERE p.dose_val_rx ~ '^[0-9]+(\\.[0-9]+)?$'
    GROUP BY a.ethnicity, p.drug
    ORDER BY a.ethnicity, TotalDose DESC
""").fetchdf()

In [ ]:
ethnicity_drug_summary

,Ethnicity,Drug,TotalDose
0,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,5% Dextrose,16900.000
1,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,Heparin,15000.000
2,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,0.9% Sodium Chloride,13601.000
3,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,Lactulose Enema,5000.000
4,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,Vancomycin,4000.000
...,...,...,...
1227,WHITE,PNEUMOcoccal Vac Polyvalent,0.500
1228,WHITE,Calcitriol,0.500
1229,WHITE,Epoprostenol Na,0.300
1230,WHITE,Hyoscyamine,0.125


In [ ]:
con.sql(
    """
    SELECT dose_val_rx FROM prescriptions
    """
)

┌────────────────────────┐
│      dose_val_rx       │
│        varchar         │
├────────────────────────┤
│ 0.5                    │
│ 10                     │
│ 10                     │
│ 1                      │
│ 100                    │
│ 5000                   │
│ 20                     │
│ 162                    │
│ 10                     │
│ 100                    │
│  ·                     │
│  ·                     │
│  ·                     │
│ 25-50                  │
│ 2                      │
│ 7.5                    │
│ 650                    │
│ 40                     │
│ 650                    │
│ 650                    │
│ 87.5                   │
│ 62.5                   │
│ 1000                   │
├────────────────────────┤
│         ? rows         │
│ (>9999 rows, 20 shown) │
└────────────────────────┘

In [5]:
import duckdb

query = """
WITH cleaned AS (
  SELECT
    drug,
    ethnicity,
    -- remove commas and lowercase everything
    lower(replace(dose_val_rx, ',', '')) AS dose_str
  FROM prescriptions
  JOIN admissions USING (hadm_id)
  WHERE dose_val_rx IS NOT NULL
),

ranges AS (
  SELECT
    drug,
    ethnicity,
    -- extract ranges like "10-20" and take average
    CASE
      WHEN dose_str ~ '^[0-9.]+-[0-9.]+$' THEN
        (CAST(split_part(dose_str, '-', 1) AS DOUBLE) + CAST(split_part(dose_str, '-', 2) AS DOUBLE)) / 2
      WHEN dose_str ~ '^[0-9.]+$' THEN
        CAST(dose_str AS DOUBLE)
      ELSE NULL
    END AS dose_avg
  FROM cleaned
),

filtered AS (
  SELECT * FROM ranges
  WHERE dose_avg IS NOT NULL
),

agg AS (
  SELECT
    drug,
    SUM(CASE WHEN ethnicity LIKE '%ASIAN%' THEN dose_avg ELSE 0 END) AS asian_total,
    SUM(CASE WHEN ethnicity LIKE '%WHITE%' THEN dose_avg ELSE 0 END) AS white_total,
    SUM(CASE WHEN ethnicity LIKE '%BLACK%' THEN dose_avg ELSE 0 END) AS black_total,
    SUM(CASE WHEN ethnicity LIKE '%HISPANIC%' THEN dose_avg ELSE 0 END) AS hispanic_total,
    SUM(CASE WHEN ethnicity LIKE '%OTHER%' THEN dose_avg ELSE 0 END) AS other_total,
    SUM(CASE WHEN ethnicity LIKE '%UNKNOWN%' THEN dose_avg ELSE 0 END) AS unknown_total,
    SUM(CASE WHEN ethnicity LIKE '%UNABLE%' THEN dose_avg ELSE 0 END) AS unable_total,
    SUM(CASE WHEN ethnicity LIKE '%AMERICAN INDIAN%' OR ethnicity LIKE '%ALASKA%' THEN dose_avg ELSE 0 END) AS native_total
  FROM filtered
  GROUP BY drug
),

with_totals AS (
  SELECT *,
    asian_total + white_total + black_total + hispanic_total + other_total + unknown_total + unable_total + native_total AS total
  FROM agg
)

SELECT
  drug,
  asian_total,
  white_total,
  black_total,
  hispanic_total,
  other_total,
  unknown_total,
  unable_total,
  native_total,
  total,
  ROUND(asian_total / total * 100, 1) AS asian_pct,
  ROUND(white_total / total * 100, 1) AS white_pct,
  ROUND(black_total / total * 100, 1) AS black_pct,
  ROUND(hispanic_total / total * 100, 1) AS hispanic_pct,
  ROUND(other_total / total * 100, 1) AS other_pct,
  ROUND(unknown_total / total * 100, 1) AS unknown_pct,
  ROUND(unable_total / total * 100, 1) AS unable_pct,
  ROUND(native_total / total * 100, 1) AS native_pct
FROM with_totals
ORDER BY total DESC
LIMIT 50;

"""

# Run the query
result = duckdb.query(query).to_df()
result


,drug,asian_total,white_total,black_total,hispanic_total,other_total,unknown_total,unable_total,native_total,total,asian_pct,white_pct,black_pct,hispanic_pct,other_pct,unknown_pct,unable_pct,native_pct
0,Heparin Sodium,0.0,1450000.00,150000.0,175000.0,0.0,25000.0,0.0,0.0,1800000.00,0.0,80.6,8.3,9.7,0.0,1.4,0.0,0.0
1,Heparin,15000.0,469600.00,45200.0,51800.0,5000.0,15000.0,5000.0,15000.0,621600.00,2.4,75.5,7.3,8.3,0.8,2.4,0.8,2.4
2,Nystatin,0.0,0.00,0.0,500000.0,0.0,0.0,0.0,0.0,500000.00,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
3,Ny,0.0,500000.00,0.0,0.0,0.0,0.0,0.0,0.0,500000.00,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.9% Sodium Chloride,0.0,131239.00,11355.0,54967.5,2000.0,1500.0,14800.0,13601.0,229462.50,0.0,57.2,4.9,24.0,0.9,0.7,6.4,5.9
5,NS,7000.0,134200.00,5650.0,3200.0,4750.0,8100.0,0.0,100.0,163000.00,4.3,82.3,3.5,2.0,2.9,5.0,0.0,0.1
6,Vitamin D,400.0,154800.00,800.0,400.0,0.0,0.0,0.0,0.0,156400.00,0.3,99.0,0.5,0.3,0.0,0.0,0.0,0.0
7,D5W,11000.0,87485.00,11950.0,3250.0,1000.0,14100.0,1300.0,200.0,130285.00,8.4,67.1,9.2,2.5,0.8,10.8,1.0,0.2
8,5% Dextrose,0.0,45510.00,5300.0,36050.0,1200.0,0.0,4250.0,16900.0,109210.00,0.0,41.7,4.9,33.0,1.1,0.0,3.9,15.5
9,Vancomycin,0.0,60500.00,13500.0,27750.0,0.0,0.0,2000.0,4000.0,107750.00,0.0,56.1,12.5,25.8,0.0,0.0,1.9,3.7


In [6]:
query = """
WITH cleaned AS (
  SELECT
    drug,
    ethnicity,
    lower(replace(dose_val_rx, ',', '')) AS dose_str
  FROM prescriptions
  JOIN admissions USING (hadm_id)
  WHERE dose_val_rx IS NOT NULL
),

parsed AS (
  SELECT
    drug,
    ethnicity,
    CASE
      WHEN dose_str ~ '^[0-9.]+-[0-9.]+$' THEN
        (CAST(split_part(dose_str, '-', 1) AS DOUBLE) + CAST(split_part(dose_str, '-', 2) AS DOUBLE)) / 2
      WHEN dose_str ~ '^[0-9.]+$' THEN
        CAST(dose_str AS DOUBLE)
      ELSE NULL
    END AS dose_avg
  FROM cleaned
),

filtered AS (
  SELECT * FROM parsed WHERE dose_avg IS NOT NULL
),

aggregated AS (
  SELECT
    ethnicity,
    drug,
    SUM(dose_avg) AS total_dose
  FROM filtered
  GROUP BY ethnicity, drug
),

ranked AS (
  SELECT *,
    RANK() OVER (PARTITION BY ethnicity ORDER BY total_dose DESC) AS rnk
  FROM aggregated
)

SELECT
  ethnicity AS "Ethnicity",
  drug AS "Top Drug",
  total_dose AS "Total Dose"
FROM ranked
WHERE rnk = 1
ORDER BY ethnicity;
"""

result = duckdb.query(query).to_df()
result

,Ethnicity,Top Drug,Total Dose
0,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,5% Dextrose,16900.0
1,ASIAN,Heparin,15000.0
2,BLACK/AFRICAN AMERICAN,Heparin Sodium,150000.0
3,HISPANIC OR LATINO,5% Dextrose,19950.0
4,HISPANIC/LATINO - PUERTO RICAN,Nystatin,500000.0
5,OTHER,Esmolol,5000.0
6,OTHER,Heparin,5000.0
7,UNABLE TO OBTAIN,0.9% Sodium Chloride,14800.0
8,UNKNOWN/NOT SPECIFIED,Epoetin Alfa,80000.0
9,WHITE,Heparin Sodium,1450000.0


**Question 2**

In [7]:
duckdb.sql("CREATE TABLE patients AS SELECT * FROM read_csv_auto('PATIENTS.csv', DATEFORMAT='%Y-%m-%d')")
duckdb.sql("CREATE TABLE admissions AS SELECT * FROM read_csv_auto('ADMISSIONS.csv', DATEFORMAT='%Y-%m-%d %H:%M:%S')")

CatalogException: Catalog Error: Table with name "patients" already exists!

In [8]:
query = """
SELECT
  p.subject_id,
  p.dob,
  a.admittime,
  CAST(DATEDIFF('day', p.dob, a.admittime) / 365.25 AS INTEGER) AS age
FROM patients p
JOIN admissions a
  ON p.subject_id = a.subject_id
WHERE EXTRACT(YEAR FROM p.dob) >= 1900
"""
result = duckdb.query(query).to_df()
result

,subject_id,dob,admittime,age
0,10006,2094-03-05,2164-10-23 21:09:00,71
1,10011,2090-06-05,2126-08-14 22:32:00,36
2,10013,2038-09-03,2125-10-04 23:36:00,87
3,10017,2075-09-21,2149-05-26 17:19:00,74
4,10019,2114-06-20,2163-05-14 20:43:00,49
...,...,...,...,...
115,44083,2057-11-15,2112-05-22 15:37:00,55
116,44083,2057-11-15,2112-05-28 15:45:00,55
117,44212,2078-06-16,2123-11-24 14:14:00,45
118,44222,2107-06-27,2180-07-19 06:55:00,73


In [9]:
result.to_csv('result.csv', index=False)

from google.colab import files
files.download('result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = """
WITH patients_with_age AS (
  SELECT
    p.subject_id,
    CAST(DATEDIFF('day', p.dob, a.admittime) / 365.25 AS INTEGER) AS age
  FROM patients p
  JOIN admissions a ON p.subject_id = a.subject_id
  WHERE EXTRACT(YEAR FROM p.dob) >= 1900
),
procedures_with_age AS (
  SELECT
    pr.subject_id,
    pr.icd9_code,
    CASE
      WHEN pa.age <= 19 THEN '<=19'
      WHEN pa.age BETWEEN 20 AND 49 THEN '20–49'
      WHEN pa.age BETWEEN 50 AND 79 THEN '50–79'
      ELSE '80+'
    END AS age_group
  FROM procedures_icd pr
  JOIN patients_with_age pa ON pr.subject_id = pa.subject_id
),
procedure_titles AS (
  SELECT
    pwa.age_group,
    d.long_title,
    COUNT(*) AS count
  FROM procedures_with_age pwa
  LEFT JOIN d_icd_procedures d ON pwa.icd9_code = d.icd9_code
  GROUP BY pwa.age_group, d.long_title
),
ranked AS (
  SELECT *,
         RANK() OVER (PARTITION BY age_group ORDER BY count DESC) AS rnk
  FROM procedure_titles
)
SELECT age_group, long_title, count
FROM ranked
WHERE rnk <= 3
ORDER BY age_group, count DESC;
"""

results = duckdb.query(query).to_df()
results

,age_group,long_title,count
0,20–49,"Venous catheterization, not elsewhere classified",12
1,20–49,Enteral infusion of concentrated nutritional s...,11
2,20–49,Insertion of endotracheal tube,9
3,20–49,Continuous invasive mechanical ventilation for...,9
4,50–79,"Venous catheterization, not elsewhere classified",184
5,50–79,Enteral infusion of concentrated nutritional s...,170
6,50–79,Insertion of endotracheal tube,51
7,80+,"Venous catheterization, not elsewhere classified",17
8,80+,Transfusion of packed cells,16
9,80+,Insertion of endotracheal tube,9


**Question 3**

In [ ]:
import pandas as pd

In [ ]:
query = """
SELECT
  p.gender,
  AVG(i.los) AS Average_LengthofStay
FROM icustays i
JOIN patients p ON i.subject_id = p.subject_id
WHERE i.los IS NOT NULL
GROUP BY p.gender
"""

results = duckdb.query(query).to_df()
results

,gender,Average_LengthofStay
0,F,5.540071
1,M,3.513830


In [ ]:
query = """
SELECT
  a.ethnicity,
  AVG(i.los) AS avg_los
FROM icustays i
JOIN admissions a ON i.subject_id = a.subject_id
WHERE i.los IS NOT NULL
GROUP BY a.ethnicity
ORDER BY avg_los DESC
"""

results = duckdb.query(query).to_df()
results

,ethnicity,avg_los
0,UNABLE TO OBTAIN,13.357000
1,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,11.337150
2,HISPANIC OR LATINO,7.459633
3,BLACK/AFRICAN AMERICAN,6.867700
4,UNKNOWN/NOT SPECIFIED,4.510662
5,WHITE,4.123055
6,ASIAN,3.890050
7,HISPANIC/LATINO - PUERTO RICAN,3.243067
8,OTHER,0.926067
